<a href="https://colab.research.google.com/github/antonpolishko/colab-notebooks-sink/blob/master/task-risk/Exposure_json_files_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

task-risk
https://trello.com/c/lGTIDqwq

# Importing Key Libraries

In [0]:
import os
import glob
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from tqdm.notebook import tqdm

# Reading in the data

In [0]:
os.chdir('C:\\Users\\Cafral\\Desktop\\kaggle\\CORD-19-research-challenge\\2020-03-13\\') #change this to your relevnt directory
root_path = 'C:\\Users\\Cafral\\Desktop\\kaggle\\CORD-19-research-challenge\\2020-03-13'  #change this to your relevnt directory
all_json_paths = glob.glob(f'{root_path}\\**\\*.json', recursive=True)
len(all_json_paths)

13202

In [0]:
#source:https://www.kaggle.com/amogh05/cord-19-eda-question-topic-modeling-starter
#add more vars as required

class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.title = content['metadata']['title']
            self.abstract = []
            self.body_text = []
            self.biblio = []
            self.biblio_doi = []
            self.img_tables = []
            self.back_matter = []
            
            
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])          
            self.body_text = '\n'.join(self.body_text)
            
            # bibliography
            for bib_id, details in content['bib_entries'].items():
                self.biblio.append(details['title'])
                self.biblio_doi.append(details['other_ids'])
            self.biblio = '\n'.join(self.biblio)
            #self.biblio_doi = '\n'.join(self.biblio_doi)
            
            #img and table references
            for ref_id,details in content['ref_entries'].items():
                self.img_tables.append(details['text'])
            self.img_tables = '\n'.join(self.img_tables)
            
            #back_matter
            for entry in content['back_matter']:
                self.back_matter.append(entry['text'])
            self.back_matter = '\n'.join(self.back_matter)
            
    def __repr__(self):
        return f'{self.paper_id}:{self.title}-{self.abstract}... {self.body_text}...{self.biblio}...{self.img_tables}...{self.back_matter}'
        
    
dict_ = {'paper_id': [],'title':[], 'abstract': [], 'body_text': [],'biblio':[],'bidoi':[],'img_tables':[]}
for idx, entry in enumerate(all_json_paths):
    if idx % (len(all_json_paths) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json_paths)}')
    #print(entry)
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['title'].append(content.title)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    dict_['biblio'].append(content.biblio)   
    dict_['img_tables'].append(content.img_tables)  
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text','biblio','img_tables'])
df_covid.head()

#identify dups
df_covid.describe(include='all')

df_covid.drop_duplicates(['abstract'], inplace=True)
df_covid.describe(include='all')

Processing index: 0 of 13202
Processing index: 1320 of 13202
Processing index: 2640 of 13202
Processing index: 3960 of 13202
Processing index: 5280 of 13202
Processing index: 6600 of 13202
Processing index: 7920 of 13202
Processing index: 9240 of 13202
Processing index: 10560 of 13202
Processing index: 11880 of 13202
Processing index: 13200 of 13202


,paper_id,abstract,body_text,biblio,img_tables
count,11113,11113,11113,11113,11113
unique,11113,11113,11113,11110,10788
top,7e6fc218bd723f70d305eadff23e036875076250,Infection of Lewis rats with the murine corona...,Infection & Immunity Medical Investigation Uni...,"Ebola by the numbers: The size, spread and cos...",
freq,1,1,1,2,326


In [0]:
df_covid['all_text'] = df_covid['abstract'] + '' + df_covid['body_text'] 

# Find synonyms

In [0]:
#This approach does not  work well:  defining list mannually better
import nltk 
from nltk.corpus import wordnet 
synonyms = [] 

  
for syn in wordnet.synsets('exposure'): 
    for l in syn.lemmas(): 
        synonyms.append(l.name()) 
        if l.antonyms(): 
            antonyms.append(l.antonyms()[0].name())
print(set(synonyms))

{'picture', 'photo', 'photograph', 'exposure', 'pic', 'vulnerability'}


In [0]:
#defining list mannually better
stage_syn_list = ['exposure','vulnerability','vulnerable'] 

# Filter By Stage

In [0]:
disease_stage_list = ['exposure' ,'acquisition' ,'progression', 'development' ,'complications' ,'fatality', 'disability']

In [0]:
def filterByStage(text,stage_syn_list):
    paper_list =[]
    
    for idx_num,row in text.iterrows():
        for stage in stage_syn_list:
            stage_found = False
            if stage in row.all_text.split():
                stage_found = True
            else:
                pass 
        if stage_found==True:
            paper_list.append(row.all_text)
    return paper_list

In [0]:
stage_dict = {}

stage = disease_stage_list[0]

stage_dict[stage] = filterByStage(df_covid,stage_syn_list)

In [0]:
#for later ease while searching for relevant papers
exposure = pd.DataFrame(stage_dict[stage])

# NLP Starts

In [0]:
import spacy
nlp = spacy.load('C:\\Users\\Cafral\\Desktop\\kaggle\\en_core_sci_lg-0.2.4\\en_core_sci_lg-0.2.4\\en_core_sci_lg\\en_core_sci_lg-0.2.4')

# We also need to detect language, or else we'll be parsing non-english text 
# as if it were English. 
from spacy_langdetect import LanguageDetector
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

nlp.max_length=2000000

# New stop words list 
customize_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'fig', 'fig.', 'al.',
    'di', 'la', 'il', 'del', 'le', 'della', 'dei', 'delle', 'una', 'da',  'dell',  'non', 'si'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

# LDA : Kaggle Notebook Approach
https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles

In [0]:
def spacy_tokenizer(sentence):
    return [word.lemma_ for word in nlp(sentence) if not (word.like_num or word.is_stop or word.is_punct or word.is_space or len(word)==1)] 
    # remove numbers (e.g. from references [1], etc.)

### Create vector representation of data

In [0]:
tf_vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, max_features=800000) 
tf = tf_vectorizer.fit_transform(tqdm(stage_dict[stage]))

print(tf.shape)

import joblib
joblib.dump(tf_vectorizer, 'tf_vectorizer.csv')
joblib.dump(tf, 'tf.csv')

KeyboardInterrupt: 

### LDA

In [0]:
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(tf)
joblib.dump(lda_tf, 'lda.csv')

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

### Discover Topics

In [0]:
tfidf_feature_names = tf_vectorizer.get_feature_names()

def print_top_words(model, vectorizer, n_top_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
    
print_top_words(lda_tf, tfidf_feature_names, 25)


Topic #0: virus infection sample study human viral animal disease respiratory bat specie test case detect high mers-cov sequence pathogen host include report antibody positive strain result

Topic #1: case disease health study datum model infection risk outbreak number transmission time contact report population include individual estimate high rate control influenza public result increase

Topic #2: cell infection mouse response expression virus immune viral gene level increase study lung show cytokine induce control effect result activation disease day type macrophage production

Topic #3: cat pedv group study effect increase treatment level gene cell calf cancer pig high disease animal piglet expression result tissue stress control compare liver show

Topic #4: cell virus protein viral membrane rna fusion particle show mm structure infection result replication domain contain entry activity mutant ph residue indicate mutation observe min

Topic #5: antibody cell protein vaccine acti

### Create Topic Distance Matrix

In [0]:
topic_dist = pd.DataFrame(lda_tf.transform(tf))
topic_dist.to_csv('topic_dist.csv', index=False)

In [0]:
topic_dist.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,...,0.000015,0.000015,0.011124,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015
1,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,0.000005,0.027063,0.000005,0.000838,0.437342,0.000005,0.000005,0.000005,0.000005,0.000005
2,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.010219,0.000010,0.449822,0.000010,0.000010,0.000010,0.000010,0.000010
3,0.000009,0.000009,0.000009,0.000009,0.000009,0.222896,0.000009,0.000009,0.647718,0.008317,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
4,0.029781,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,...,0.000035,0.000035,0.000035,0.285344,0.000035,0.000035,0.000035,0.000035,0.000035,0.041184


### Get Paper Related to Stage of Disease

In [0]:
# Need synonyms especially here

In [0]:
#get most similar paper
from scipy.spatial import distance
def get_k_nearest_docs(doc_dist, k=5, lower=1950, upper=2020, only_covid19=False, get_dist=False):
    '''
    doc_dist: topic distribution (sums to 1) of one article
    
    Returns the index of the k nearest articles (as by Jensen–Shannon divergence in topic space). 
    '''
    
    #relevant_time = df.publish_year.between(lower, upper)
    
   # if only_covid19:
   #     is_covid19_article = df.body_text.str.contains('COVID-19|SARS-CoV-2|2019-nCov|SARS Coronavirus 2|2019 Novel Coronavirus') #TODO: move outside
   #     topic_dist_temp = topic_dist[relevant_time & is_covid19_article]
   #     
   # else:
    #    topic_dist_temp = topic_dist[relevant_time]
    
    distances = topic_dist.apply(lambda x: distance.jensenshannon(x, doc_dist), axis=1)
    k_nearest = distances[distances != 0].nsmallest(n=k).index
    
    if get_dist:
        k_distances = distances[distances != 0].nsmallest(n=k)
        return k_nearest, k_distances
    else:
        return k_nearest
    
#d = get_k_nearest_docs(topic_dist[1].iloc[0],k=10)

In [0]:
def relevant_articles(df,tasks, k=3, lower=1950, upper=2020, only_covid19=False):
    tasks = [tasks] if type(tasks) is str else tasks 
    
    tasks_tf = tf_vectorizer.transform(tasks)
    tasks_topic_dist = pd.DataFrame(lda_tf.transform(tasks_tf))

    for index, bullet in enumerate(tasks):
        print(bullet)
        recommended = get_k_nearest_docs(tasks_topic_dist.iloc[index], k, lower, upper, only_covid19)
        print(list(recommended))
        recommended = df.iloc[recommended] #stage_dict[stage][','.join(list(recommended))]#
    return recommended

In [0]:
task = ['exposure']
relevant_articles(exposure,task,k=3) #k is the number of relevant articles

exposure
[475, 406, 1450]


,0
475,The International Consortium for Prevention an...
406,Aerobiology plays a fundamental role in the tr...
1450,Background: Because of the high global prevale...
